<h1>Webscraping Postal Codes of Canada</h1>
<h2>Description</h2>

We Download the contents of the web page and we use get to download the contents of the webpage in text format and store in a variable called data:

In [57]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd

In [58]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib")  # create a soup object using the variable 'data'

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
We add each cell data to the dataframe <b> pc_canada 

In [60]:
pc_canada = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

for row in soup.find('tbody').find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        PostalCode =col[0].text.replace("\n","")
        Borough =col[1].text.replace("\n","")
        Neighborhood =col[2].text.replace("\n","")
    
        pc_canada = pc_canada.append({"PostalCode":PostalCode, "Borough":Borough, "Neighborhood":Neighborhood}, ignore_index=True)
        #print("{}, {}, {}".format(PostalCode,Borough,Neighborhood))
        
#pc_canada - #Uncomment this cell to check the dataframe

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [61]:
for index, row in pc_canada.iterrows():
    if row['Borough'] == "Not assigned":
        pc_canada = pc_canada.drop([index])
        
pc_canada = pc_canada.reset_index(drop=True)
#pc_canada - #Uncomment this cell to check the dataframe

If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
(Since there are no such cells we don't see any changes)

In [62]:
for index, row in pc_canada.iterrows():
    if row['Neighborhood'] == "Not assigned":
        print(row['Neightborhood'])
        row['Neighborhood'] = row['Borough']
        
#pc_canada - #Uncomment this cell to check the dataframe

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table. (Since they are already combined we don't see any changes)

In [63]:
pc_canada['Neighborhood'] = pc_canada.groupby(['PostalCode'])['Neighborhood'].transform(lambda x : ', '.join(x)) 
#pc_canada - #Uncomment this cell to check the dataframe

We check the dimensions of the dataframe pc_canada

In [64]:
pc_canada.shape

(103, 3)